In [1]:
## Install the package if needed:
#install.packages('Mcomp', repos='http://cran.us.r-project.org')

## Load the relevent packages
library(ncdf4)  # For reading in the NCEP wind fields
#library(R.matlab)  # If you need to read in matlab .mat files
library(openxlsx)  # If you need to read in .xlsx files
#library(rNOMADS)  # For reading grib2 data files (NOMADS data for instance)
#library(rGDAL)  #
library(RColorBrewer)
library(compiler)  # required for JIT (below)

## Enable compilation (speed gain?)
enableJIT(3)

## Helper function for converting the date time stamps.
conv_excel_time = function(x, tz='US/Pacific') {
    as.POSIXct(as.Date(x,origin="1899-12-30", tz=tz))
}

get.qual.pal = function(n=100, pal='Accent') {
    colorRampPalette(brewer.pal(8, pal))(n)
}

get.seq.pal = function(n=100, pal='YlOrRd') {
    colorRampPalette(rev(brewer.pal(11, pal)))(n)
}

get.div.pal = function(n=100, pal='Spectral') {
    colorRampPalette(rev(brewer.pal(11, pal)))(n)
}

make.div.pal = function(x=100, n, pal='Spectral') {
    get.div.pal(n, pal=pal)[as.numeric(cut(x, breaks = n))]
}

[1] 3

---
# 3. Determine MLD
This section reads in the CTD downcast files and identifies the 0.03 density change to calculate the MLD. THe MLD is then linearly interpolated onto the ship's track.

In [ ]:
## Read in the data files and convert the excel time
ctd.log = read.xlsx('./Raw Data/CTD/DataZoo - P1706 CTD Cast Log.xlsx', sheet = 'Data')
ctd.data = read.xlsx('./Raw Data/CTD/DataZoo - P1706 CTD Downcast Data.xlsx')

str(ctd.data)

In [ ]:
## this section calculates the mld for each cast
mld = data.frame(Lat = 0, Long = 0, mld = 0)
mld.times = c()

## Go through each cast and determine the MLD and then save that depth to a table.
for (i in unique(ctd.data$Cast)){
    
    ## Select the relevent density profile
    l = which(ctd.data$Cast == i)
    rho = ctd.data$Sigma_00[l]
    
    if (length(rho) > 10) { ## make sure it was a real cast (i.e. more than 10 data points)
        
        l.log = which(ctd.log$Cast == i)  # Which entry is this cast in the ctd log?
        rho.10 = rho[ctd.data$Pressure[l] == 10]  # density at 10 meters depth
        ll = which.min((rho - (rho.10 + 0.03))^2)  # What is the closest density to rho10 + 0.03?
        #ll = which.min((rho - (rho.10 + 0.125))^2)  # What is the closest density to rho10 + 0.03?
        
        # Save the Data
        mld = rbind(mld, c(ctd.log$Lat[l.log], ctd.log$Long[l.log], ctd.data$Pressure[l[ll]]) )
        mld.times = c(mld.times, ctd.data$DateTime_GMT[l[ll]])
    }
}
mld.times = conv_excel_time(mld.times)
mld = mld[-1,]

In [ ]:
par(mfrow=c(2,2))
#mld.old = mld$mld
hist(mld.old, breaks = 20)
hist(mld$mld, breaks = 20)
plot(mld.old - mld$mld)

## Save the results

In [ ]:
save(mld, file='./RStates/mld.rdata')
save(mld.times, file='./RStates/mld.times.rdata')